In [ ]:
using Gridap
using Gridap.Algebra;
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData

using GridapGmsh

using LinearAlgebra

using FillArrays
using SparseArrays

# Geometry

In [ ]:
ri = 19.1e-3;   # Cable inner radius
ro = 37.5e-3;   # Cable outer radius

lc1 = 2e-3;      # Mesh density at outside of cable
lc2 = 0.1e-3;   # Mesh density at the conductor edge

In [ ]:
gmsh.finalize()
gmsh.initialize()

gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("cable_skin")

# Points
gmsh.model.geo.addPoint(0, 0, 0, lc1, 1);
gmsh.model.geo.addPoint(ri, 0, 0, lc2, 2);
gmsh.model.geo.addPoint(ro, 0, 0, lc1, 3);
gmsh.model.geo.addPoint(0, ri, 0, lc2, 4);
gmsh.model.geo.addPoint(0, ro, 0, lc1, 5);

# Lines
gmsh.model.geo.addLine(1, 2, 1);
gmsh.model.geo.addLine(2, 3, 2);
gmsh.model.geo.addLine(1, 4, 3);
gmsh.model.geo.addLine(4, 5, 4);
gmsh.model.geo.addCircleArc(2, 1, 4, 5);
gmsh.model.geo.addCircleArc(3, 1, 5, 6);

# Surfaces
gmsh.model.geo.addCurveLoop([1, 5, -3], 1)
gmsh.model.geo.addCurveLoop([2, 6, -4, -5], 2)

gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.geo.addPlaneSurface([2], 2)

# Physics
#  Dirichlet boundary on outside of cable
gmsh.model.addPhysicalGroup(0, [3, 5], 1)
gmsh.model.setPhysicalName(0, 1, "D1p")
gmsh.model.addPhysicalGroup(1, [6], 1)
gmsh.model.setPhysicalName(1, 1, "D1")

#  Neumann boundary on two insides
gmsh.model.addPhysicalGroup(0, [1, 2, 4], 2)
gmsh.model.setPhysicalName(0, 2, "N1p")
gmsh.model.addPhysicalGroup(1, [1, 2, 3, 4], 2)
gmsh.model.setPhysicalName(1, 2, "N1")

#  Material groups
gmsh.model.addPhysicalGroup(2, [1], 1)
gmsh.model.setPhysicalName(2, 1, "conductor")
gmsh.model.addPhysicalGroup(2, [2], 2)
gmsh.model.setPhysicalName(2, 2, "insulator")

# Generate mesh
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("geo/hv_cable.msh")

# Problem Setup

In [ ]:
I = 1000;             # Current in the cable [A]

mu0 = 4*pi*1e-7;      # Permeability of vacuum
f = 50;               # Frequency of the time-harmonic current
omega = 2*pi*f;      

sigma_cond = 3.69e7;  # Conductivity of aluminium

# Gridap Setup

In [ ]:
# Define norm of a complex vector for Gridap to use on the CellField which results from Bh = ∇(uh)
norm_complex(a::VectorValue{2, ComplexF64}) = sqrt(norm(a[1])^2 + norm(a[2])^2)
norm_complex(a::CellField) = Operation(norm_complex)(a)

# Gridap Implementation

In [ ]:
# Load geometry from gmsh
model = GmshDiscreteModel("geo/hv_cable.msh")

In [ ]:
# Create a reference Lagrangian finite element space to use for the construction of test space V0
order = 2
reffe = LagrangianRefFE(Float64, Polytope(Fill(TET_AXIS, 2)...), order)

# Define the test space Vh and trial space Uh
Vh = FESpace(model, reffe; conformity=:H1, dirichlet_tags = ["D1", "D1p"], vector_type=Vector{ComplexF64})
Uh = TrialFESpace(Vh, [0, 0])

# Extract the triangulation from the model and create a quadrature on the cells (should be >2*order for exact integration of the mass matrix)
Th = Triangulation(model)
Qh = CellQuadrature(Th, 2*order)

In [ ]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Th)

const tag_cond = get_tag_from_name(labels, "conductor");

function fsource(tag)
    return 0.0;
end

function fsigma(tag)
    if tag == tag_cond
        return sigma_cond;
    else
        return 0.0;
    end
end

function fmur(tag)
    return 1.0;
end

function fnu(tag)
    return 1 / (mu0 * fmur(tag))
end


In [ ]:
dv = get_fe_basis(Vh);
du = get_trial_fe_basis(Uh);

# Calculate contributions of the cells to the linear system
cellvals_A = ∫( (fnu ∘ τ) ⋅ ∇(du) ⋅ ∇(dv) - 1im * omega * (fsigma ∘ τ) ⋅ du ⋅ dv )*Qh;
cellvals_f = ∫( (fsource ∘ τ) ⋅ dv )*Qh;

# Assemble the linear system
assem = SparseMatrixAssembler(Uh, Vh)
σk = get_cell_dof_ids(Uh);

rs = ([cellvals_f], [σk]);
b = allocate_vector(assem, rs);
assemble_vector!(b, assem, rs);

rs = ([cellvals_A], [σk], [σk]);
A = allocate_matrix(assem, rs);
assemble_matrix!(A, assem, rs);

In [ ]:
# Extend the linear system with an additional row and column per circuit equation
N  = num_free_dofs(Uh);
nc = 1;

C = Matrix(UniformScaling(1), nc, nc);
b2 = [I / 4];

# Calculate the cell contributions to the extra vectors
Thc = Triangulation(model, tags = "conductor")
Qhc = CellQuadrature(Thc, 2*order)
cellvals_BT = ∫( (4/(pi*ri^2)) ⋅ dv )*Qhc;
cellvals_B  = ∫( (1im * omega * (fsigma ∘ τ)) ⋅ dv )*Qhc;

# Assemble the circuit coupling equations into two extra vectors
σck = get_cell_dof_ids(Uh, Thc);

rs = ([cellvals_BT], [σck])
BT = allocate_vector(assem, rs)
assemble_vector!(BT, assem, rs);

rs = ([cellvals_B], [σck])
B = allocate_vector(assem, rs)
assemble_vector!(B, assem, rs);

In [ ]:
# Create the modified system \tilde{A} = [A BT; B C] and \tilde{b} = [b1; b2]
At = [A BT; transpose(B) C];
bt = [b; b2];

# Solve the modified linear system and package the solution in a FEFunction
u = At \ bt;
uh = FEFunction(Uh, u);

In [ ]:
# Post-processing for magnetic field and current density
Bh = norm_complex(gradient(uh));

function fsource(tag)
    if tag == tag_cond
        return 4 * u[end] / (pi * ri^2);
    else
        return 0.0;
    end
end

J0    = fsource ∘ τ; # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

In [ ]:
writevtk(Th, "images/hvcable_gridap/cable_50Hz", cellfields=["Az"=>abs(uh), "normB" => Bh, "normJ" => abs(J0 + Jeddy)])

At $f = 0\,\mathrm{Hz}$, the current is distributed uniformly. This is as expected, because there is no skin effect.
![Current Density: DC](images/hvcable_gridap/normJ_0Hz.png)

At $f = 50\,\mathrm{Hz}$, the current begins to move outward due to the increasing skin effect.
![Current Density: 50Hz](images/hvcable_gridap/normJ_50Hz.png)

At $f = 1\,\mathrm{kHz}$, most of the current now flows along the surface of the conductor. The middle barely participates in the conduction process. The skin depth is approximately $\delta = 2.6\,\mathrm{mm}$.
![Current Density: 1kHz](images/hvcable_gridap/normJ_1kHz.png)